# 多步骤筛选算法分析
## Sequential Screening Algorithm for Advanced Liver Fibrosis

参考文献: Chen et al. 2024 - US-based Sequential Algorithm Integrating an AI Model for Advanced Liver Fibrosis

### 主要功能:
1. 两步筛选算法 (Clinical → Deep Learning model)
2. 桑基图展示患者流向 (TN/FN/FP/TP)
3. PPV/NPV 随患病率变化曲线 (贝叶斯定理)
4. 诊断性能评估
5. 支持手动调整阈值和预测概率

In [ ]:
# 导入脚本
from stepwise_screening import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 配置显示
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
%matplotlib inline

print("导入完成!")

## 1. 配置参数

在这里修改模型标签、数据路径等配置

In [ ]:
# 创建配置
config = Config(
    pred_path="all_models_sample_predictions.csv",
    output_dir="stepwise_outputs_v3",
    
    # 数据集分割名称
    split_val="Val",
    split_internal_test="Test",
    split_external_test="External",
    
    # 模型标签 (必须与CSV中的Model列匹配)
    tag_m1="Radiomics_Only",              # M1: Echo-Net / DL模型
    tag_m2="Radiomics_plus_A",            # M2: DL + 临床A参数
    tag_m3="Radiomics_plus_AllClinical",  # M3: DL + 所有临床参数
    tag_m4="ClinicalA_Only",              # M4: 临床A参数 (类似FIB-4)
    tag_m5="BaseClinical_Only",           # M5: 基础临床特征
)

print("配置完成!")
print(f"数据路径: {config.pred_path}")
print(f"输出目录: {config.output_dir}")

## 2. 加载数据

In [ ]:
# 加载预测数据
wide = load_predictions(config.pred_path, config)

print(f"总样本数: {len(wide)}")
print(f"\n数据集分布:")
print(wide['Split'].value_counts())

# 查看各数据集的患病率
print("\n各数据集患病率:")
for split in wide['Split'].unique():
    subset = wide[wide['Split'] == split]
    prevalence = subset['TrueLabel'].mean()
    print(f"  {split}: n={len(subset)}, 患病率={prevalence:.2%}")

print("\n数据预览:")
wide.head()

## 3. 在验证集上选择阈值 (Youden's J)

In [ ]:
# 获取验证集数据
val_data = wide[wide['Split'] == config.split_val]
y_val = val_data['TrueLabel'].values

print(f"验证集样本数: {len(val_data)}")
print(f"验证集患病率: {y_val.mean():.2%}")

# 为每个模型选择最优阈值
thresholds = {}
print("\n最优阈值 (Youden's J):")
print("-" * 50)

for tag in config.all_model_tags:
    if tag in val_data.columns:
        thr, youden_j = find_threshold_youden(y_val, val_data[tag].values)
        thresholds[tag] = thr
        display_name = config.model_display_names.get(tag, tag)
        print(f"{display_name:20s}: threshold = {thr:.4f}, Youden J = {youden_j:.4f}")

print("-" * 50)

## 4. ⚠️ 手动调整阈值 (可选)

如果需要手动调整阈值，在下面的单元格中取消注释并修改

In [ ]:
# ============================================================
# 手动调整阈值 - 取消注释并修改以下值
# ============================================================

# thresholds[config.tag_m1] = 0.05   # Echo-Net
# thresholds[config.tag_m2] = 0.60   # Echo-Net+A
# thresholds[config.tag_m3] = 0.55   # Echo-Net+All
# thresholds[config.tag_m4] = 0.40   # Clinical-A
# thresholds[config.tag_m5] = 0.45   # Clinical-Base

print("当前使用的阈值:")
for tag, thr in thresholds.items():
    display_name = config.model_display_names.get(tag, tag)
    print(f"  {display_name}: {thr:.4f}")

## 5. 在测试集上评估

In [ ]:
# 选择评估的数据集
cohort_name = "External_Test"  # 可选: "Internal_Test", "External_Test", "Combined_Test"

if cohort_name == "Internal_Test":
    test_data = wide[wide['Split'] == config.split_internal_test]
elif cohort_name == "External_Test":
    test_data = wide[wide['Split'] == config.split_external_test]
else:  # Combined
    test_data = wide[wide['Split'].isin([config.split_internal_test, config.split_external_test])]

y_test = test_data['TrueLabel'].values
print(f"评估数据集: {cohort_name}")
print(f"样本数: {len(test_data)}, 患病率: {y_test.mean():.2%}")

In [ ]:
# 定义两步筛选策略 (Step1 → Step2)
strategies = {
    "Two-step (Clinical-A → Echo-Net+All)": (config.tag_m4, config.tag_m3),
    "Two-step (Clinical-Base → Echo-Net+All)": (config.tag_m5, config.tag_m3),
    "Two-step (Clinical-A → Echo-Net)": (config.tag_m4, config.tag_m1),
    "Two-step (Clinical-Base → Echo-Net)": (config.tag_m5, config.tag_m1),
}

# 评估所有策略
results = {}

print("\n" + "="*80)
print(f"评估结果 - {cohort_name}")
print("="*80)

# 多步骤策略
for strategy_name, (step1_tag, step2_tag) in strategies.items():
    result = stepwise_two_stage(
        y_true=y_test,
        p_step1=test_data[step1_tag].values,
        thr_step1=thresholds[step1_tag],
        p_step2=test_data[step2_tag].values,
        thr_step2=thresholds[step2_tag]
    )
    results[strategy_name] = result
    
    print(f"\n{strategy_name}")
    print(f"  Step1通过: {result.n_pass_step1}/{len(y_test)} ({100*result.n_pass_step1/len(y_test):.1f}%)")
    print(f"  最终性能:")
    print(f"    Sensitivity: {result.final_metrics['Sensitivity']:.3f}")
    print(f"    Specificity: {result.final_metrics['Specificity']:.3f}")
    print(f"    PPV: {result.final_metrics['PPV']:.3f}")
    print(f"    NPV: {result.final_metrics['NPV']:.3f}")
    print(f"    Accuracy: {result.final_metrics['Accuracy']:.3f}")

# 单模型基线
print("\n" + "-"*40)
print("单模型基线:")
print("-"*40)

for tag in config.all_model_tags:
    y_prob = test_data[tag].values
    y_pred = (y_prob >= thresholds[tag]).astype(int)
    metrics = compute_metrics(y_test, y_pred, y_prob)
    
    display_name = config.model_display_names.get(tag, tag)
    print(f"\n{display_name}:")
    print(f"  Sens={metrics['Sensitivity']:.3f}, Spec={metrics['Specificity']:.3f}, "
          f"PPV={metrics['PPV']:.3f}, NPV={metrics['NPV']:.3f}, Acc={metrics['Accuracy']:.3f}")

## 6. 绘制桑基图

In [ ]:
# 选择要绘制的策略
strategy_to_plot = "Two-step (Clinical-A → Echo-Net+All)"

if strategy_to_plot in results:
    result = results[strategy_to_plot]
    
    # 绘制桑基图
    os.makedirs(f"{config.output_dir}/sankey", exist_ok=True)
    plot_sankey_matplotlib(
        result.zone_counts,
        f"{strategy_to_plot}\n{cohort_name}",
        f"{config.output_dir}/sankey/sankey_{cohort_name}_interactive.png"
    )
    
    # 显示流向统计
    print("患者流向统计:")
    for key, value in result.zone_counts.items():
        print(f"  {key}: {value}")
    
    # 显示图片
    from IPython.display import Image, display
    display(Image(filename=f"{config.output_dir}/sankey/sankey_{cohort_name}_interactive.png"))

## 7. PPV/NPV vs 患病率曲线

In [ ]:
# 收集所有策略的Sens/Spec用于绘图
strategies_for_plot = {}

# 多步骤策略
for name, result in results.items():
    strategies_for_plot[name] = result.final_metrics

# 单模型
for tag in config.all_model_tags:
    y_prob = test_data[tag].values
    y_pred = (y_prob >= thresholds[tag]).astype(int)
    metrics = compute_metrics(y_test, y_pred, y_prob)
    display_name = f"Single ({config.model_display_names.get(tag, tag)})"
    strategies_for_plot[display_name] = metrics

# 绘制PPV/NPV曲线
os.makedirs(f"{config.output_dir}/curves", exist_ok=True)
plot_ppv_npv_vs_prevalence(
    strategies_for_plot,
    f"PPV and NPV vs Disease Prevalence\n{cohort_name}",
    f"{config.output_dir}/curves/ppv_npv_interactive.png"
)

# 显示图片
from IPython.display import Image, display
display(Image(filename=f"{config.output_dir}/curves/ppv_npv_interactive.png"))

## 8. 汇总表格

In [ ]:
# 创建汇总表格
summary_rows = []

# 多步骤策略
for name, result in results.items():
    summary_rows.append({
        "Strategy": name,
        "Sensitivity": result.final_metrics['Sensitivity'],
        "Specificity": result.final_metrics['Specificity'],
        "PPV": result.final_metrics['PPV'],
        "NPV": result.final_metrics['NPV'],
        "Accuracy": result.final_metrics['Accuracy'],
        "Referral_Rate": result.final_metrics['Positive_Rate'],
        "Step2_Entry": f"{result.n_pass_step1}/{len(y_test)}",
    })

# 单模型
for tag in config.all_model_tags:
    y_prob = test_data[tag].values
    y_pred = (y_prob >= thresholds[tag]).astype(int)
    metrics = compute_metrics(y_test, y_pred, y_prob)
    display_name = f"Single ({config.model_display_names.get(tag, tag)})"
    
    summary_rows.append({
        "Strategy": display_name,
        "Sensitivity": metrics['Sensitivity'],
        "Specificity": metrics['Specificity'],
        "PPV": metrics['PPV'],
        "NPV": metrics['NPV'],
        "Accuracy": metrics['Accuracy'],
        "Referral_Rate": metrics['Positive_Rate'],
        "Step2_Entry": "-",
    })

summary_df = pd.DataFrame(summary_rows)
summary_df = summary_df.round(3)

print(f"\n汇总表格 - {cohort_name}:")
print("="*100)
display(summary_df)

# 保存到CSV
os.makedirs(f"{config.output_dir}/tables", exist_ok=True)
summary_df.to_csv(f"{config.output_dir}/tables/summary_interactive_{cohort_name}.csv", index=False)
print(f"\n已保存到: {config.output_dir}/tables/summary_interactive_{cohort_name}.csv")

## 9. 运行完整分析 (一键执行)

In [ ]:
# 运行完整分析 - 这将生成所有图表和表格
overall_results, step_results = run_analysis(config)

print("\n完整结果预览:")
overall_results.head(10)